v0: DBPIA, Scienceon Rule계산(Yop + co_authorship + keyword)
v1: title 조건 추가, jaro_winkler_metric 추가
v2: site 조건 변경
v3: Rule계산(joc + yop + title_cossim + abstract_cossim) 추가, TF-IDF 추가, 교신저자 비교
v4: co_authorship 비율추가
v5: site 조건 수정, 함수/변수이름 변경, filter 함수 추가, title_cossim + abstract_cossim 삭제

In [5]:
from bson.objectid import ObjectId
from pymongo import MongoClient
import numpy as np
from numpy import dot
from numpy.linalg import norm
import jaro
from sklearn.feature_extraction.text import TfidfVectorizer

def filter(site, rawdata):
    if site == 'NTIS' :
        inst = rawdata['ldAgency']
        coauthor = rawdata['rsc'].split(";")
        year = int(rawdata['prdStart'][:4])
        keyword = rawdata['koKeyword'].split(",")
        journal = ""
        conference = ""
        title = ""
        mng = rawdata['mng']

    else :
        inst = rawdata['author_inst'].split(";")[rawdata['author'].split(";").index(name)]
        coauthor = rawdata['author'].split(";")[1:-1]
        year = int(rawdata['issue_year'][:4])
        keyword = rawdata['paper_keyword'].replace(" ", "").split(".")
        journal = rawdata['journal']
        conference = rawdata['issue_inst']
        title = rawdata['title']
        mng = rawdata['author'].split(";")[-1]
        #abstract = rawdata['abstract']

    return inst, coauthor, year, keyword, journal, conference, title, mng
    #return inst, coauthor, year, keyword, journal, conference, title, mng, abstract

def Secondary_filter(name, site1, raw_one1, site2, raw_one2):
    inst1, coauthor1, year1, keyword1, journal1, conference1, title1, mng1 = filter(site1, raw_one1)
    inst2, coauthor2, year2, keyword2, journal2, conference2, title2, mng2 = filter(site2, raw_one2)
    print(inst1, inst2)
    
    inst = jaro.jaro_winkler_metric(inst1, inst2)
    print(inst)
    weight = 0
    joc = 0
    
    if name in coauthor1:
        coauthor1.remove(name)

    if name in coauthor2:
        coauthor2.remove(name)

    co_author_count = len([i for i in coauthor1 if i in coauthor2])

    if site1 != 'NTIS' and site2 != 'NTIS' :
        if title1 == title2  or inst >= 0.8:
            weight = 4
            return weight

        else:
            joc = 1 if journal1 == journal2 and conference1 == conference2 else -1
    else:
        if inst >= 0.8:
            weight = 3
            return weight
            
    yop = -(2*(abs(year1-year2)/10)-1)
            
    if len(coauthor1) > len(coauthor2):
        co_author_ratio = co_author_count/len(coauthor1)
    else:
        co_author_ratio = co_author_count/len(coauthor2)
                
    co_authorship = (1 - np.exp(-co_author_count))/2 + (co_author_ratio/2)
    keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

    print(f'joc: {joc} | yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword}')

    weight = joc + yop + co_authorship + keyword

    return weight

In [7]:
client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')

name = '김희숙'
site1 = 'KCI'
site2 = 'KCI'
input_paper1 = ObjectId('61ce98c10f91d3fd2b7cab65')
input_paper2 = ObjectId('61ce98ce0f91d3fd2b7cab7f')

if site1 == 'Scienceon' :
    site_raw1 = client['SCIENCEON']['Rawdata']
elif site1 == 'NTIS' :
    site_raw1 = client['NTIS']['Rawdata']
elif site1 == 'KCI' :
    site_raw1 = client['KCI']['Rawdata']

if site2 == 'Scienceon' :
    site_raw2 = client['SCIENCEON']['Rawdata']
elif site2 == 'NTIS' :
    site_raw2 = client['NTIS']['Rawdata']
elif site2 == 'KCI' :
    site_raw2 = client['KCI']['Rawdata']

input_raw1 = site_raw1.find_one({ '_id' : input_paper1 })
input_raw2 = site_raw2.find_one({ '_id' : input_paper2 })

Secondary_filter(name, site1, input_raw1, site2, input_raw2)

광주과학기술원 전남대학교
0.44761904761904764
joc: 1 | yop: 1.0 | co_authorship: 0.0 | keyword: 0.6321205588285577


2.6321205588285577